In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import logging

from conda_forge_tick.utils import setup_logging

setup_logging("INFO")

In [3]:
from conda_forge_tick.make_graph import make_graph
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

names = ["flit"]

gx = load_graph()

gx = make_graph(names, gx, debug=True)

2024-03-21 08:15:28,914 INFO     conda_forge_tick.make_graph || reading graph
2024-03-21 08:15:28,915 INFO     conda_forge_tick.make_graph || start feedstock fetch loop
2024-03-21 08:15:30,003 INFO     conda_forge_tick.make_graph || feedstock fetch loop completed
2024-03-21 08:15:30,004 INFO     conda_forge_tick.make_graph || inferring nodes and edges
2024-03-21 08:15:43,556 INFO     conda_forge_tick.make_graph || new nodes and edges inferred
2024-03-21 08:15:43,558 INFO     conda_forge_tick.make_graph || memory usage: svmem(total=25769803776, available=7481769984, percent=71.0, used=10210246656, free=71581696, active=7421116416, inactive=7266156544, wired=2789130240)


In [46]:
import pprint
import yaml

print(yaml.dump(gx.nodes["pre-commit"]["payload"].data["requirements"]))

build: []
host:
- python
- pip
run:
- nodeenv
- cfgv
- pyyaml
- identify
- pre-commit
- python
- virtualenv
test:
- pip



In [79]:
print(sorted(gx.predecessors("bcrypt")))

['c_compiler_stub', 'pip', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust']


In [ ]:
def _filter_stubby_and_ignored_nodes(graph, ignored_packages):
    """remove any stub packages and ignored packages from the graph

    **operates in place**
    """
    # filter out stub packages and ignored packages
    for node, attrs in list(graph.nodes("payload")):
        if (
            (not attrs)
            or node.endswith("_stub")
            or node.startswith("m2-")
            or node.startswith("m2w64-")
            or node.startswith("__")
            or (node in ignored_packages)
            or all_noarch(attrs)
        ):
            pluck(graph, node)
    # post-plucking cleanup
    graph.remove_edges_from(nx.selfloop_edges(graph))

## Check Some Version Stuff

In [ ]:
from conda_forge_tick.update_upstream_versions import update_upstream_versions

In [ ]:
import copy

gxc = copy.deepcopy(gx)

for node in list(gxc.nodes):
    if node not in ["tzdata", "openssl", "jpeg", "cddlib"]:
        gxc.remove_node(node)

In [ ]:
update_upstream_versions(gxc, debug=True)

## Look at a Migration

In [7]:
from conda_forge_tick.auto_tick import migration_factory, add_arch_migrate

In [8]:
mgs = []
add_arch_migrate(mgs, gx)

making aarch64+ppc64le and osx-arm64 migrations


In [9]:
mg = mgs[0]

In [88]:
from conda_forge_tick.make_graph import get_deps_from_outputs_lut

arches = {
    "linux_aarch64": "default",
    "linux_ppc64le": "default",
}


graph2 = nx.create_empty_copy(gx)
for node, attrs in gx.nodes(data="payload"):
    for plat_arch in arches:
        deps = set().union(
            *attrs.get(
                f"{plat_arch}_requirements",
                attrs.get("requirements", {}),
            ).values()
        )
        for dep in get_deps_from_outputs_lut(deps, gx.graph["outputs_lut"]):
            graph2.add_edge(dep, node)

In [89]:
print(sorted(graph2.predecessors("bcrypt")))

['c_compiler_stub', 'pip', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust']


In [90]:
from conda_forge_tick.utils import pluck

graph = graph2
packages = mg.target_packages.copy()
for target in mg.target_packages:
    if target in graph.nodes:
        packages.update(nx.ancestors(graph, target))
for node in list(graph.nodes.keys()):
    if node not in packages:
        pluck(graph, node)
# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

In [91]:
print(sorted(graph2.predecessors("bcrypt")))

['c_compiler_stub', 'pip', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust']


In [92]:
from conda_forge_tick.migrators.migration_yaml import all_noarch

ignored_packages = {
    "make",
    "perl",
    "toolchain",
    "posix",
    "patchelf",  # weird issue
}

latest_preds = sorted(graph.predecessors("bcrypt"))

for node, attrs in list(graph.nodes("payload")):
    if (
        (not attrs)
        or node.endswith("_stub")
        or node.startswith("m2-")
        or node.startswith("m2w64-")
        or node.startswith("__")
        or (node in ignored_packages)
        or all_noarch(attrs)
    ):
        pluck(graph, node)
        new_latest_preds = sorted(graph.predecessors("bcrypt"))
        if new_latest_preds != latest_preds:
            print(node)
            print("    new preds:", new_latest_preds)
            print("    old preds:", latest_preds)
            print(" ")
            latest_preds = new_latest_preds

# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

pip
    new preds: ['c_compiler_stub', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust', 'wheel']
    old preds: ['c_compiler_stub', 'pip', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust']
 
pytest
    new preds: ['c_compiler_stub', 'exceptiongroup', 'hatchling', 'iniconfig', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust', 'setuptools_scm', 'tomli', 'wheel']
    old preds: ['c_compiler_stub', 'pytest', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust', 'wheel']
 
setuptools
    new preds: ['c_compiler_stub', 'exceptiongroup', 'hatchling', 'iniconfig', 'python', 'rust_compiler_stub', 'setuptools-rust', 'setuptools_scm', 'tomli', 'wheel']
    old preds: ['c_compiler_stub', 'exceptiongroup', 'hatchling', 'iniconfig', 'python', 'rust_compiler_stub', 'setuptools', 'setuptools-rust', 'setuptools_scm', 'tomli', 'wheel']
 
setuptools_scm
    new preds: ['c_compiler_stub', 'exceptiongroup', 'hatchling', 'iniconfig

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mg.graph["outputs_lut"]

In [10]:
import networkx as nx

In [69]:
print(sorted(mg.graph.predecessors("bcrypt")))

['aiohttp', 'argon2-cffi-bindings', 'astroid', 'backports.zoneinfo', 'black', 'brotli', 'chktex', 'coverage', 'cryptography', 'debugpy', 'falcon', 'greenlet', 'importlib_metadata', 'importlib_resources', 'jsonpointer', 'jupyter-packaging', 'jupyter_core', 'lazy-object-proxy', 'markupsafe', 'matplotlib', 'mypy', 'nodejs', 'numpy', 'pandas', 'pandoc', 'pdm', 'psutil', 'psycopg2', 'pydantic-core', 'pynacl', 'pyobjc-framework-cocoa', 'pyqt', 'python', 'pywin32', 'pywinpty', 'pyyaml', 'pyzmq', 'r-languageserver', 'regex', 'rpds-py', 'ruamel.yaml', 'simplejson', 'texlab', 'tornado', 'trio', 'tzlocal', 'ujson', 'uvloop', 'watchdog']


In [ ]:
mgs = []
add_rebuild_broken_migrator(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mmctx.effective_graph.nodes

In [ ]:
mg_name = "libffi33"

mgs = []
migration_factory(mgs, gx, only_keep=[mg_name])

for i in range(len(mgs)):
    if mgs[i].name == mg_name:
        break
mg = mgs[i]

In [ ]:
import copy

attrs = copy.deepcopy(mg.graph.nodes["python"]["payload"].data)

attrs["branch"] = "3.6"
mg.filter(attrs)

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)


In [ ]:
mmctx.effective_graph.nodes

## Check the Status Report

In [ ]:
from conda_forge_tick.status_report import  graph_migrator_status

In [ ]:
out2, build_sequence, gv = graph_migrator_status(mg, mg.graph)

In [ ]:
gv.view()

In [ ]:
build_sequence

In [ ]:
len(build_sequence)

In [ ]:
gx.nodes["proj"]["payload"].data